In [1]:
import pandas as pd
import numpy as np
from gensim import models, corpora
import pickle
from ast import literal_eval

In [2]:
dropbox = "/Users/mr/Dropbox/moviemeta/"

In [69]:
documents = np.load(dropbox + 'imdb_plots_since_2014_d2v.pickle')

In [ ]:
sents=[]
for doc in documents:
    for sent in doc:
        sents.append(sent)
d2v = models.Doc2Vec(sents, dbow_words=1)
# should really be an iterator but couldn't get it to work
#def itersents(docs):
#    for doc in docs:
#        for sent in doc:
#            yield sent

In [83]:
movie = sents[0][1]
print movie
d2v.docvecs.most_similar(movie)

['#1 at the Apocalypse Box Office (2015)']


[('Tracing Outlines (2015)', 0.20295196771621704),
 ('Warped! (2016)', 0.19414910674095154),
 ('We Vault (2015)', 0.18911390006542206),
 ('Um Homem S\xf3 (2015)', 0.1787264347076416),
 ('Effloresce (2015)', 0.17776648700237274),
 ('10 Milliarden (2015)', 0.17670276761054993),
 ('Glamour Boys of NYC (2015)', 0.17613360285758972),
 ('Zero Organism (2016)', 0.17454853653907776),
 ('Sanskriti (2015)', 0.17271117866039276),
 ('The Book of Emmanuel (2015)', 0.17206630110740662)]